<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

## _*Gate Errors*_ 


***
### Contributors
David McKay

## Introduction

The general form of an arbitrary single qubit gate is a rotation $\theta$ around some axis in the Bloch sphere with unit vector $\hat{n}$ such that
$$U(\theta,\hat{n}) = e^{-i \theta \hat{n} \cdot \sigma} $$
Physically these rotations are performed by applying some oscillating electromagnetic field at the qubit frequency to the location of the qubit. It is not necessary to calibrate all such rotations, only a few rotation angles and axes are enough. At minimum we need a $\pi/2$ rotation ($\theta=\pi/4$) along two orthogonal axis. So then the calibration question becomes how to tell that the gate $X90p$ ($\pi/2$ around $X$) is really $ e^{-i \frac{\pi}{4} \sigma_X} $?

Here we will look at different sequences to amplify possible errors in the gate.


**Contents**

[Amplitude](#sect1)

[Phase](#sect2)


### References

[1]<a id="ref1"></a> David C. McKay, Christopher J. Wood, Sarah Sheldon, Jerry M. Chow and Jay M. Gambetta. Efficient Z-Gates for Quantum Computing. https://arxiv.org/abs/1612.00858

Code imports
==============

In [1]:
# Import general libraries (needed for functions)
import numpy as np
import time

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.tools.visualization import plot_histogram

# Import measurement calibration functions
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,
                                                       CompleteMeasFitter, MeasurementFilter)

<a id='sect1'></a>

# Amplitude Error

Consider a gate which is a X90p, but with a slight rotation error. 
$$U = e^{-i (\pi/2+d\theta) \sigma_X/2}  $$
One way to measure $d\theta$ would be to apply the gate starting with the qubit in the ground state and look at the $|0\rangle$ and $|1\rangle$ counts. However, the signal could be lost in the noise. To amplify the roration error we continually apply the gate in groups of 2,
$$U = X90p-[X90p-X90p]^n$$

To predict the outcome we expand using Euler's formula,
$$X90p = \text{cos}\left(\frac{\pi}{4}+\frac{d\theta}{2}\right)I - \text{sin}\left(\frac{\pi}{4}+\frac{d\theta}{2}\right) \sigma_X $$
$$X90p^{2n+1} = \text{cos}\left(n\frac{\pi}{2}+nd\theta+\frac{\pi}{4}+\frac{d\theta}{2}\right)I - \text{sin}\left(n\frac{\pi}{2}+nd\theta+\frac{\pi}{4}+\frac{d\theta}{2}\right) \sigma_X $$
and the population in the excited state is
$$ P_{|1\rangle, 2n+1} = \text{sin}^2\left(n\frac{\pi}{2}+nd\theta+\frac{\pi}{4}+\frac{d\theta}{2}\right) $$ 
$$ P_{|1\rangle, 2n+1} = \frac{1}{2}-\frac{1}{2}\text{cos}\left(n\pi+2nd\theta+\frac{\pi}{2}+d\theta\right) $$ 
$$ P_{|1\rangle, 2n+1} = \frac{1}{2}+\frac{(-1)^n}{2}\text{sin}\left(2nd\theta+d\theta\right) $$ 
for small $d\theta$ this amplifies the error by $n$,
$$ P_{|1\rangle, 2n+1} \approx \frac{1}{2}+\frac{(-1)^n d\theta}{2}(2n+1) $$ 


<a id='sect2'></a>

# Phase Error

A phase error means that the angle between the X90p and Y90p is not perfectly $\pi/2$. We assume the amplitude is correct so,
$$X90p = e^{-i \frac{\pi}{4}\sigma_X}$$
$$Y90p = e^{-i \frac{\pi}{4}\left[\text{cos}(d\phi)\sigma_Y+\text{sin}(d\phi)\sigma_X\right]}$$
The angle error can be amplified by the sequence
$$X90p-[\{Y90p\}^2 \{X90p\}^2]^n - Y90p$$